## **Import Libraries**

In [ ]:
import stat
import os
import os.path


## **OS commands automations**

In [ ]:
print(os.name) 

In [ ]:
dir(os)

In [ ]:
print("\nCurrent Working Directory: ", os.getcwd())


In [ ]:
# To print absolute path on your system 
print("\n\n Absolute OS Path: ", os.path.abspath('.'))


In [ ]:
print("\n\n Relative OS Path: ", os.path.relpath('data'))

In [ ]:
# To print files and directories in the current directory 
# on your system 
print("\n\nList of files in current directory: ")
os.listdir('.')
os.listdir('./sample_data')


## **Working with compressed files**

##### Compressing Individual Files

In [ ]:
f = open(os.path.join("./", 'file1.txt'), 'w')
f.write('This is the new file.')
f.close()

f = open(os.path.join("./", 'file2.txt'), 'w')
f.write('This is the new file 2.')
f.close()

f = open(os.path.join("./", 'file3.txt'), 'w')
f.write('This is the new file 3.')
f.close()

f = open(os.path.join("./", 'file4.txt'), 'w')
f.write('This is the new file 4.')
f.close()

f = open(os.path.join("./", 'file5.txt'), 'w')
f.write('This is the new file 5.')
f.close()

In [ ]:
import zipfile

jungle_zip = zipfile.ZipFile('Test File1.zip', 'w')
jungle_zip.write('./file1.txt', compress_type=zipfile.ZIP_DEFLATED)
 
jungle_zip.close()

#### Compressing Multiple Files

In [ ]:
import os
import zipfile
 
fantasy_zip = zipfile.ZipFile('Test Files.zip', 'w')
 
for folder, subfolders, files in os.walk('.'):
 
    for file in files:
        if file.endswith('.txt'):
            fantasy_zip.write(os.path.join(folder, file), os.path.relpath(os.path.join(folder,file), '.'), compress_type = zipfile.ZIP_DEFLATED)
 
fantasy_zip.close()

#### Extracting All Files

In [ ]:
import zipfile
         
fantasy_zip = zipfile.ZipFile('Test Files.zip')
fantasy_zip.extractall('.')
 
fantasy_zip.close()

#### Extracting Individual Files

In [ ]:
import zipfile
         
fantasy_zip = zipfile.ZipFile('Test Files.zip')
fantasy_zip.extract('file1.txt', '.')
 
fantasy_zip.close()

## **Search for files and show permissions in a friendly format**

##### Take input from user

In [ ]:
strPath = input("Specify Path:")

strPattern = input("Specify Search pattern:")

##### Get File permissions in the path specified

In [ ]:
for path in os.listdir(strPath):
    print (path)
    if os.access( path, os.R_OK): # is path readable?
        print ("  - readable")
    else:
        print ("  - unreadable")
    
    if os.access( path, os.W_OK): # is path writable?
        print ("  - writable")
    else:
        print ("  - read-only")

    if os.access( path, os.X_OK): # is path executable?
        print ("  - executable")
    else:
        print ("  - non-executable")



##### Get file permission in nested directories based on the file name pattern

In [ ]:
for dirpath, dirnames, filenames in os.walk(strPath):
    for filename in [f for f in filenames if strPattern in f]:
        filePath = os.path.join(dirpath, filename)
        print (filePath,":", oct(stat.S_IMODE(os.stat(filePath).st_mode)))

## **Perform operations on a tar archive**

#### How to create an uncompressed tar archive from a list of filenames:

In [ ]:
import tarfile
tar = tarfile.open("sample.tar", "w")
for name in ["file1.txt", "file2.txt"]:
    tar.add(name)
tar.close()

#### How to read a gzip compressed tar archive and display some member information:

In [ ]:
import tarfile
tar = tarfile.open("sample.tar", "r")
for tarinfo in tar:
    print (tarinfo.name, "is", tarinfo.size, "bytes in size and is",)
    if tarinfo.isreg():
        print ("a regular file.")
    elif tarinfo.isdir():
        print ("a directory.")
    else:
        print ("something else.")
tar.close()

## **Check for a running process and show information**

#### Check if a process is running

In [ ]:
import psutil
 
def checkIfProcessRunning(processName):
    '''
    Check if there is any running process that contains the given name processName.
    '''
    #Iterate over the all the running process
    for proc in psutil.process_iter():
        try:
            # Check if process name contains the given name string.
            if processName.lower() in proc.name().lower():
                return True
        except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
            pass
    return False;

In [ ]:
strProcessname = input("Process name please: ")
if checkIfProcessRunning(strProcessname):
    print(f'Yes a {strProcessname} process was running')
else:
    print(f'No {strProcessname} process was running')

#### Find PID (Process ID) of a running process by Name

As there may be many running instances of a given process. So, we will iterate over all the running process and for each process whose name contains the given string, we will keep it’s info in a list i.e.

In [ ]:
 def findProcessIdByName(processName):
    '''
    Get a list of all the PIDs of a all the running process whose name contains
    the given string processName
    '''
 
    listOfProcessObjects = []
 
    #Iterate over the all the running process
    for proc in psutil.process_iter():
       try:
           pinfo = proc.as_dict(attrs=['pid', 'name', 'create_time'])
           # Check if process name contains the given name string.
           if processName.lower() in pinfo['name'].lower() :
               listOfProcessObjects.append(pinfo)
       except (psutil.NoSuchProcess, psutil.AccessDenied , psutil.ZombieProcess) :
           pass
 
    return listOfProcessObjects;

In [ ]:
# Find PIDs od all the running instances of process that contains 'chrome' in it's name
import time
#if strProcessname == "":
strProcessname = input("Process name please: ")

listOfProcessIds = findProcessIdByName(strProcessname)
 
if len(listOfProcessIds) > 0:
   print('Process Exists | PID and other details are')
   for elem in listOfProcessIds:
       processID = elem['pid']
       processName = elem['name']
       processCreationTime =  time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(elem['create_time']))
       print((processID ,processName,processCreationTime ))
else :
   print('No Running Process found with given text')

#### Get List of all running process sorted by Highest Memory Usage

In [ ]:
import psutil
def getListOfProcessSortedByMemory():
  '''
  Get list of running process sorted by Memory Usage
  '''
  listOfProcObjects = []
  # Iterate over the list
  for proc in psutil.process_iter():
    try:
      # Fetch process details as dict
      pinfo = proc.as_dict(attrs=['pid', 'name', 'username'])
      pinfo['vms'] = proc.memory_info().vms / (1024 * 1024)
      # Append dict to list
      listOfProcObjects.append(pinfo);
    except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
      pass
  # Sort list of dict by key vms i.e. memory usage
  listOfProcObjects = sorted(listOfProcObjects, key=lambda procObj: procObj['vms'], reverse=True)
  return listOfProcObjects

In [ ]:
getListOfProcessSortedByMemory()